In [1]:
from sklearn.preprocessing import OneHotEncoder

In [2]:
from sklearn.neighbors import KDTree
import numpy as np
import pandas as pd
from FE import *

In [3]:
dict_label2id = {
    'Excellent':0, 'Good':1, 'Pass':2, 'Fail':3
}

In [4]:
tr_data = pd.read_csv('./rawdata/first_round_training_data.csv', encoding='utf-8')

In [5]:
te_data = pd.read_csv('./rawdata/first_round_testing_data.csv', encoding='utf-8')

In [6]:
tr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 21 columns):
Parameter1       6000 non-null float64
Parameter2       6000 non-null float64
Parameter3       6000 non-null float64
Parameter4       6000 non-null float64
Parameter5       6000 non-null float64
Parameter6       6000 non-null float64
Parameter7       6000 non-null float64
Parameter8       6000 non-null float64
Parameter9       6000 non-null float64
Parameter10      6000 non-null float64
Attribute1       6000 non-null float64
Attribute2       6000 non-null float64
Attribute3       6000 non-null float64
Attribute4       6000 non-null float64
Attribute5       6000 non-null float64
Attribute6       6000 non-null float64
Attribute7       6000 non-null float64
Attribute8       6000 non-null float64
Attribute9       6000 non-null float64
Attribute10      6000 non-null float64
Quality_label    6000 non-null object
dtypes: float64(20), object(1)
memory usage: 984.5+ KB


In [7]:
tr_data.describe()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10
count,6000.000000,6.000000e+03,6000.000000,6000.000000,6.000000e+03,6.000000e+03,6000.000000,6000.000000,6.000000e+03,6000.000000,6.000000e+03,6.000000e+03,6000.000000,6.000000e+03,6.000000e+03,6.000000e+03,6.000000e+03,6.000000e+03,6.000000e+03,6.000000e+03
mean,485.198954,1.952379e+03,406.348529,189.205842,5.868163e+00,5.598597e+00,272.300821,22.133231,5.823664e+04,35.902066,5.351032e+04,8.842895e+04,202.732522,6.271156e+03,1.285985e+06,3.400877e+03,3.166863e+04,4.545394e+05,7.860735e+05,1.989520e+07
std,11401.971393,5.419588e+04,12018.898941,2621.779049,8.631865e+00,7.686898e+00,2027.460514,302.172768,3.173068e+06,101.629348,1.894866e+06,2.854543e+06,2754.503943,2.585508e+05,9.850723e+07,1.999508e+05,1.116889e+06,2.847995e+07,5.885757e+07,1.529785e+09
min,0.000003,1.683797e-07,0.000005,0.000002,9.985942e-07,9.810610e-07,0.000624,0.023699,3.962831e-04,0.001024,6.457597e-10,5.493777e-09,0.000001,4.344229e-06,1.635864e-06,5.456653e-05,4.411611e-03,3.011145e-03,2.366726e-03,1.974795e-03
25%,0.088221,4.765771e-02,0.102304,0.118314,7.416296e-01,3.818154e-01,0.152059,0.035407,5.930812e-01,0.101478,3.100601e-02,3.173294e-02,0.098178,1.319373e-01,1.150362e-01,1.443224e-01,1.135895e-01,1.115708e-01,1.267717e-01,1.253268e-01
50%,1.050304,9.405962e-01,1.044908,1.068467,2.225808e+00,2.684398e+00,0.600827,2.931083,5.930812e-01,0.727611,9.449563e-01,1.030499e+00,0.945947,8.297855e-01,7.539155e-01,9.864325e-01,5.891701e-01,5.922906e-01,6.644951e-01,6.705953e-01
75%,10.938565,1.569199e+01,10.665397,9.902064,6.680183e+00,7.117767e+00,2.374035,17.850021,6.783967e+00,13.969790,2.784766e+01,3.001000e+01,9.803828,6.052952e+00,5.736029e+00,7.422284e+00,4.637157e+00,4.773050e+00,5.571728e+00,4.905501e+00
max,713682.207601,3.034568e+06,864530.632179,139767.494732,8.367195e+01,4.117541e+01,35698.591553,20086.397753,1.738083e+08,517.197610,1.197060e+08,1.615907e+08,128491.084741,1.572225e+07,7.630284e+09,1.488592e+07,7.789923e+07,2.187522e+09,4.558485e+09,1.184961e+11


In [8]:
tr_value = tr_data[['Parameter'+str(i) for i in range(1,11)]].values
te_value = te_data[['Parameter'+str(i) for i in range(1,11)]].values

In [9]:
for i in range(10):
    tr_value[:, i] = log_trans(tr_value[:, i], 2)
    te_value[:, i] = log_trans(te_value[:, i], 2) 

In [10]:
tr_label = tr_data['Quality_label'].values

In [11]:
tree = KDTree(tr_value)

In [12]:
dist, ind = tree.query(te_value, k = 10) 

In [13]:
def get_label(idx):
    l = np.zeros(4)
    for i in idx:
        l[dict_label2id[tr_label[i]]] += 1
    #l = l / sum(l)
    return l

In [14]:
submit_result = []
for i in ind:
    submit_result.append(get_label(i))

In [15]:
submit_pd = pd.read_csv('./rawdata/submit_example.csv', encoding='utf-8', index_col=None)

In [16]:
len(submit_result)

6000

In [17]:
group_value = te_data['Group'].values

In [18]:
submit_result = np.array(submit_result)

In [19]:
np.sum(submit_result[group_value == 0], axis=0)

array([130., 163., 129.,  78.])

In [20]:
final_result = []
for i in range(120):
    final_result.append(np.sum(submit_result[group_value == i], axis=0))

In [21]:
final_result = [i / sum(i) for i in final_result]

In [22]:
final_result = np.array(final_result)

In [23]:
submit_pd[['Excellent ratio', 'Good ratio', 'Pass ratio', 'Fail ratio']] = final_result

In [24]:
submit_pd.to_csv('knn_log2_10.csv', index=None)